In [1]:
#load "LoadBaseToolWPF.csx"
using BaseTool;

In [2]:
List<int> numbers = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];

## 筛选数据


In [3]:
numbers.Where( n => n % 2 == 0 ).Dump();

2, 4, 6, 8, 10


## 选择数据进行操作


In [4]:
numbers.Select(n => n * 2 ).Dump();

2, 4, 6, 8, 10, 12, 14, 16, 18, 20


## 方法链和查询语句


In [7]:
numbers.Where(n => n > 5).Select(n=>n).Dump();
(from item in numbers where item > 5 select item).Dump();

6, 7, 8, 9, 10


## orderby 和 descending 进行排序


In [12]:
(from item in numbers orderby item descending select item).Dump();

10, 9, 8, 7, 6, 5, 4, 3, 2, 1


## ThenBy / ThenByDescending: 用于在现有排序的基础上进行次级排序


## SelectMany 用于将嵌套的集合展平，并选择元素。


In [13]:
public class Product
{
    public string Name { get; set; }
    public List<string> Categories { get; set; }
}

var products = new List<Product>
{
    new Product 
    { 
        Name = "Apple", 
        Categories = new List<string> { "Fruit", "Organic" } 
    },
    new Product 
    { 
        Name = "Bread", 
        Categories = new List<string> { "Food", "Bakery" } 
    }
};

products.SelectMany(p => p.Categories).Dump();
(from product in products
from category in product.Categories 
select new { product.Name, category }).Dump();

Fruit, Organic, Food, Bakery
{ Name = Apple, category = Fruit }, { Name = Apple, category = Organic }, { Name = Bread, category = Food }, { Name = Bread, category = Bakery }


## GroupBy: 用于将数据源中的元素按某个键进行分组。


In [32]:
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
}


var people = new List<Person>
{
    new Person { Name = "Alice", Age = 25 },
    new Person { Name = "Bob", Age = 25 },
    new Person { Name = "Charlie", Age = 30 },
    new Person { Name = "David", Age = 20 }
};

var ageGroups = people.GroupBy(p => p.Age);

foreach (var group in ageGroups)
{
    Console.WriteLine($"Age: {group.Key}, People: {string.Join(", ", group)}");
}

Age: 25, People: Submission#33+Person, Submission#33+Person
Age: 30, People: Submission#33+Person
Age: 20, People: Submission#33+Person


## Join / GroupJoin: 用于基于键值将两个序列连接起来。


In [41]:
public class Order
{
    public int OrderId { get; set; }
    public int CustomerId { get; set; }
}

public class Customer
{
    public int Id { get; set; }
    public string Name { get; set; }
}

var orders = new List<Order>
{
    new Order { OrderId = 10, CustomerId = 1 },
    new Order { OrderId = 20, CustomerId = 2 },
    // Order with no matching customer
    new Order { OrderId = 30, CustomerId = 3 }
};

var customers = new List<Customer>
{
    new Customer { Id = 1, Name = "Alice" },
    new Customer { Id = 2, Name = "Bob" }
};

// 连接orders列表和customers列表，orders的CustomerId与customers的Id相匹配

var orderDetails = orders.Join(
                            customers, 
                            o => o.CustomerId, 
                            c => c.Id, 
                            (o, c) => new { Order = o, CustomerName = c.Name });

foreach (var detail in orderDetails)
{
    Console.WriteLine($"OrderID: {detail.Order.OrderId}, CustomerID: {detail.Order.CustomerId}, Customer: {detail.CustomerName}");
}

OrderID: 10, CustomerID: 1, Customer: Alice
OrderID: 20, CustomerID: 2, Customer: Bob


## Aggregate: 用于将数据源中的所有元素聚合成一个单一的结果。


In [30]:
class Student
{
    public string Name { get; set; }
    public int Age { get; set; }
}

class Grade
{
    public string StudentName { get; set; }
    public int Score { get; set; }
}

var students = new List<Student>
{
    new Student { Name = "Alice", Age = 20 },
    new Student { Name = "Bob", Age = 22 },
    new Student { Name = "Charlie", Age = 21 }
};

var grades = new List<Grade>
{
    new Grade { StudentName = "Alice", Score = 90 },
    new Grade { StudentName = "Bob", Score = 85 },
    new Grade { StudentName = "Charlie", Score = 95 }
};

var result = students.Join(grades, 
                            s => s.Name, 
                            g => g.StudentName, 
                            (s, g) => new { StudentName = s.Name, Grade = g.Score });
result.Dump(); // 连接两个序列并返回一个新的序列，其中包含每个学生的成绩

var groupedResult = students.GroupBy(s => s.Age);
foreach (var group in groupedResult)
{
        Console.WriteLine($"Age: {group.Key}");
        foreach (var student in group)
        {
            Console.WriteLine($"Name: {student.Name}");
        }
        Console.WriteLine();
}

var aggregatedResult = students.Aggregate((a, b) => a.Age > b.Age ? a : b);
Console.WriteLine($"Oldest Student: {aggregatedResult.Name}");

{ StudentName = Alice, Grade = 90 }, { StudentName = Bob, Grade = 85 }, { StudentName = Charlie, Grade = 95 }
Age: 20
Name: Alice

Age: 22
Name: Bob

Age: 21
Name: Charlie

Oldest Student: Bob


## Any / All: 用于检查数据源中是否存在满足条件的元素（Any），或所有元素是否都满足条件（All）。


## Contains: 用于检查数据源是否包含特定的元素。


## DefaultIfEmpty: 用于确保即使数据源为空也能返回一个空的序列。


## Distinct: 用于返回数据源中的唯一元素。


In [28]:
List<int> numbers = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 2, 2 ];
numbers.Distinct().Dump(); // 返回唯一元素
numbers.Contains(3).Dump(); // 检查数据源是否包含特定的元素
numbers.Any(n => n > 10).Dump(); // 检查是否存在满足条件的元素
numbers.All(n => n > 0).Dump(); // 检查所有元素是否都满足条件
numbers.Where(n=>n > 10).DefaultIfEmpty(1).Dump(); // 即使数据源为空也能返回一个空的序列

1, 2, 3, 4, 5, 6, 7, 8, 9, 10
True
False
True
1


## First / FirstOrDefault: 用于获取数据源中的第一个元素或第一个满足条件的元素，如果没有找到则返回默认值。


## Last / LastOrDefault: 类似于 First，但用于获取最后一个元素。


## Count 获取数据源中的元素数量。


In [20]:
List<int> numbers = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];
numbers.First().Dump(); // 获取第一个元素
numbers.First(n => n > 5).Dump(); // 获取第一个满足条件的元素
numbers.Last().Dump(); // 获取最后一个元素
numbers.Last(n => n > 5).Dump(); // 获取最后一个满足条件的元素
numbers.Count().Dump(); // 获取元素数量

1
6
10
10
10


## Skip / SkipWhile: 用于跳过数据源中的前 N 个元素或跳过满足条件的元素。


## Take / TakeWhile: 用于获取数据源中的前 N 个元素或获取满足条件的元素。


In [19]:
List<int> numbers = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];
numbers.Skip(3).Dump(); // 跳过前3个元素
numbers.SkipWhile(n => n < 4).Dump(); // 跳过满足条件的元素
numbers.Take(3).Dump(); // 获取前3个元素
numbers.TakeWhile(n => n < 4).Dump(); // 获取满足条件的元素

4, 5, 6, 7, 8, 9, 10
4, 5, 6, 7, 8, 9, 10
1, 2, 3
1, 2, 3


## Union / Intersect / Except: 用于集合操作，返回两个序列的并集、交集或差集


In [17]:
List<int> numbers1 = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];
List<int> numbers2 = [ 5, 6, 7, 8, 9, 10, 11, 12, 13, 14 ];
numbers1.Union(numbers2).Dump();
numbers1.Intersect(numbers2).Dump();
numbers1.Except(numbers2).Dump();

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14
5, 6, 7, 8, 9, 10
1, 2, 3, 4


## let 关键字


In [34]:

var groupA = new[] { 3, 4, 5, 6 };
var groupB = new[] { 6, 7, 8, 9 };
var someInts = from a in groupA
                from b in groupB
                let sum = a + b         //在新的变量中保存结果
                where sum == 12
                select new { a, b, sum };
foreach (var a in someInts)
{
    Console.WriteLine(a);
}

{ a = 3, b = 9, sum = 12 }
{ a = 4, b = 8, sum = 12 }
{ a = 5, b = 7, sum = 12 }
{ a = 6, b = 6, sum = 12 }


In [36]:
var groupA = new[] { 3, 4, 5, 6 };
var groupB = new[] { 6, 7, 8, 9 };
var someInts = from a in groupA
                join b in groupB on a equals b
                into groupAandB
                from c in groupAandB
                select c;
foreach (var a in someInts)
{
    Console.WriteLine(a);	// 6
}

6


In [42]:
List<int> _list = new List<int> { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 };

var _num1 = _list.FirstOrDefault(n => n == 11);
_num1.Dump();

var _num2 = _list.FirstOrDefault(n => n == 11, 11);
_num2.Dump(); // 11


0
11


## FirstOrDefault

In [47]:
class student
{
    public string name { get; set; }
    public int age { get; set; }

    override public string ToString(){
        return $"name: {name}, age: {age}";
    }
}

List<student> _list = new List<student>();
_list.Add(new student { name = "张三", age = 18 });
_list.Add(new student { name = "李四", age = 20 });
_list.Add(new student { name = "王五", age = 19 });

var _num3 = _list.FirstOrDefault(n => n.age == 20);
_num3.Dump();

var _num4 = _list.FirstOrDefault(n => n.age == 21);
_num4.Dump();  // null
(_num4 is null).Dump();

var _num5 = _list.FirstOrDefault(n => n.age == 21, new student { name = "默认", age = 20 });
_num5.Dump();

name: 李四, age: 20

True
name: 默认, age: 20
